In [2]:
import pandas as pd
import numpy as np

## Importar datos

In [3]:
def indexar_df_por_fecha_diaria(ruta_datos):
    # dataframe
    df = pd.read_csv(ruta_datos)

    # poner la fecha en formato datetime
    df['Date'] = pd.to_datetime(df['Date'])
    # establecer como índice del dataframe
    df_indexado = df.set_index('Date')
    # frecuencia diaria
    df_indexado = df_indexado.asfreq('D')

    return df_indexado

In [4]:
# DATOS PRODUCCIÓN EÓLICA TERRESTRE
df_PET = indexar_df_por_fecha_diaria('data_PET.csv')

# DATOS PRODUCCIÓN EÓLICA MARINA
df_PEM = indexar_df_por_fecha_diaria('data_PEM.csv')

df_PEM.tail()

,AL,BA,BE,BG,CY,DE,DK,EE,EL,ES,...,ME,ML,NL,NO,PL,PT,RO,SE,SI,UK
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-24,0.0,0.0,0.0,996.132449,0.0,0.0,117278.557781,10246.012483,0.0,0.0,...,0.0,0.0,14452.125815,19.553677,0.0,0.0,0.0,2973.800407,0.0,143194.073582
2025-02-25,0.0,0.0,0.0,139.868968,0.0,0.0,30690.943662,15166.442421,0.0,0.0,...,0.0,0.0,2902.891778,13.363147,0.0,0.0,0.0,1704.662205,0.0,93678.147629
2025-02-26,0.0,0.0,0.0,232.975363,0.0,0.0,7185.849632,4411.162461,0.0,0.0,...,0.0,0.0,4897.299713,8.409767,0.0,0.0,0.0,493.382513,0.0,62586.870054
2025-02-27,0.0,0.0,0.0,1442.015740,0.0,0.0,5243.713839,757.210438,0.0,0.0,...,0.0,0.0,4305.494327,8.445793,0.0,0.0,0.0,284.024268,0.0,53777.987682
2025-02-28,0.0,0.0,0.0,2339.704276,0.0,0.0,11974.200641,13.629363,0.0,0.0,...,0.0,0.0,3894.840325,9.357046,0.0,0.0,0.0,220.692498,0.0,46685.760826


## Matriz coberturas por país

In [5]:
matriz_cobertura = pd.read_csv('../MATRIZ_COBERTURA_PAISES.csv', index_col=0)

# solo nos interesan países con toda la información
matriz_cobertura['Total'] = matriz_cobertura.sum(axis=1)
matriz_cobertura.head()

,DE,PHE,PHR,PET,PEM,Total
Paises,,,,,,
AL,0,0,0,0,0,0
AT,1,1,1,1,0,4
BA,1,0,0,1,0,2
BE,1,0,0,1,0,2
BG,1,0,0,1,1,3


## Producción eólica países de interés

In [6]:
lista_PAISES_INTERES = matriz_cobertura[(matriz_cobertura['Total']==4) | (matriz_cobertura['Total']==5)].index.to_list()
print(f'Países interés = {lista_PAISES_INTERES}')

Países interés = ['AT', 'CH', 'DE', 'ES', 'FI', 'FR', 'IT', 'NO', 'PT', 'RO', 'SE', 'SK']


In [10]:
dict_df_prod_eolica = {
    'PET': df_PET,
    'PEM': df_PEM,
}

# Asegúrate de que todas las series de datos tengan la misma fecha como índice
# Vamos a crear un DataFrame vacío para las fechas
fechas = df_PET.index  # Usamos el índice de uno de los DataFrames que ya tiene fechas


In [11]:
df_prod_eolic_diaria = pd.DataFrame(index=fechas)
dict_paises_coberturas = {}

# Iteramos por los países de interés para crear una serie temporal de producción total
for pais in lista_PAISES_INTERES:

    # dataframe de tipos de energía producidas por cada país
    df_pais = pd.DataFrame(index=fechas)
    
    # para cada cobertura, obtenemos la producción del país para ese día
    for cob, df_cob in dict_df_prod_eolica.items():
        if pais in df_cob.columns:  # si el país está en los datos
            df_pais[cob] = df_cob[pais]  # guardamos todas las producciones
        
        elif (pais not in df_cob.columns) and (pais in ['AT','CH','SK']): # países sin costa de interés AT, CH, SK
            # lo dejamos en blanco
            df_pais[cob] = np.nan
        
        else:
            raise ValueError(f'El país {pais} no está en la bbdd {cob}.')

    # suma diaria de produccion
    df_pais['Prod_eolic_diaria'] = df_pais.sum(axis=1)
     
    # añadimos los totales al dataframe de produccion total por países
    df_prod_eolic_diaria[pais] = df_pais['Prod_eolic_diaria']

    # guardamos el dataframe en el diccionario
    dict_paises_coberturas[pais] = df_pais


In [12]:
dict_paises_coberturas['AT'].tail()

,PET,PEM,Prod_eolic_diaria
Date,,,
2025-02-24,1124.117474,NaN,1124.117474
2025-02-25,3750.680433,NaN,3750.680433
2025-02-26,4358.394917,NaN,4358.394917
2025-02-27,2745.555005,NaN,2745.555005
2025-02-28,3118.798334,NaN,3118.798334


In [13]:
df_prod_eolic_diaria.tail()

,AT,CH,DE,ES,FI,FR,IT,NO,PT,RO,SE,SK
Date,,,,,,,,,,,,
2025-02-24,1124.117474,15.539226,582287.409690,95783.220015,27106.470082,134702.161048,2080.952473,55388.891284,98.260652,5740.948560,3019.524104,6404.666507
2025-02-25,3750.680433,135.431701,142993.901194,138396.892727,21386.135248,176666.717761,10023.318473,18044.110458,110.280475,3703.649527,1718.488630,40059.617166
2025-02-26,4358.394917,225.182491,307116.329343,42084.705472,9217.833436,151286.015800,29002.246643,11116.168161,27.463499,10129.644016,502.608923,27607.262141
2025-02-27,2745.555005,38.151518,242651.879704,14752.610595,10522.895536,61067.395280,21480.226505,8101.121944,83.666164,23664.663599,289.800505,2501.091893
2025-02-28,3118.798334,19.670934,152727.146939,75019.080253,5558.266792,78526.249660,5886.033267,11007.377839,97.477080,19646.932272,230.944554,15130.241646


In [15]:
# Crear un DataFrame vacío para la producción anual
df_prod_eolic_anual = pd.DataFrame()

# Agrupar por año y sumar la producción para cada país
df_prod_eolic_anual = df_prod_eolic_diaria.resample('Y').sum()

df_prod_eolic_anual.to_csv('calc_dato_Produccion_Eolica_Anual_MW-MW_paises_interes.csv')